In [2]:
df = spark.sql("SELECT * FROM tbl_latest_housing_news")
display(df)

StatementMeta(, dfa0c094-e2a2-46b6-a9e3-395c9836bec4, 4, Finished, Available)

SynapseWidget(Synapse.DataFrame, ca6aad0e-c7b4-4c18-8bdf-e554d4e0a584)

In [3]:
import synapse.ml.core
from synapse.ml.services import AnalyzeText

StatementMeta(, dfa0c094-e2a2-46b6-a9e3-395c9836bec4, 5, Finished, Available)

In [4]:
model = (AnalyzeText()
        .setTextCol("description")
        .setKind("SentimentAnalysis")
        .setOutputCol("response")
        .setErrorCol("error"))

StatementMeta(, dfa0c094-e2a2-46b6-a9e3-395c9836bec4, 6, Finished, Available)

In [5]:
result = model.transform(df)

StatementMeta(, dfa0c094-e2a2-46b6-a9e3-395c9836bec4, 7, Finished, Available)

In [6]:
display(result)

StatementMeta(, dfa0c094-e2a2-46b6-a9e3-395c9836bec4, 8, Finished, Available)

SynapseWidget(Synapse.DataFrame, 0702fc91-0b82-466b-b68a-0579431de3fd)

In [7]:
from pyspark.sql.functions import col

sentiment_df = result.withColumn("sentiment", col("response.documents.sentiment"))

StatementMeta(, dfa0c094-e2a2-46b6-a9e3-395c9836bec4, 9, Finished, Available)

In [8]:
display(sentiment_df)

StatementMeta(, dfa0c094-e2a2-46b6-a9e3-395c9836bec4, 10, Finished, Available)

SynapseWidget(Synapse.DataFrame, 4b2e6469-0103-4eeb-9b8e-f4d9e3cae1bf)

In [9]:
sentiment_final_df = sentiment_df.drop("error", "response")

StatementMeta(, dfa0c094-e2a2-46b6-a9e3-395c9836bec4, 11, Finished, Available)

In [10]:
display(sentiment_final_df)

StatementMeta(, dfa0c094-e2a2-46b6-a9e3-395c9836bec4, 12, Finished, Available)

SynapseWidget(Synapse.DataFrame, 239a3c73-b41d-442b-8605-7328ac382ce6)

In [11]:
from pyspark.sql.utils import AnalysisException


try:
    # Define table name
    table_name = 'Scottish_Housing_Project.tbl_sentiment_analysis'
    # Try to write the DataFrame to a Delta table
    sentiment_final_df.write.format("delta").saveAsTable(table_name)

except AnalysisException:

    print("Table already exists")

    # Create a temporary view for the DataFrame
    sentiment_final_df.createOrReplaceTempView("vw_sentiment_final_df")
    
    # Perform merge operation using SQL
    spark.sql(f"""
        MERGE INTO {table_name} AS target_table
        USING vw_sentiment_final_df AS source_view
        ON source_view.url = target_table.url
        WHEN MATCHED AND (
            source_view.title <> target_table.title OR
            source_view.description <> target_table.description OR
            source_view.category <> target_table.category OR
            source_view.image <> target_table.image OR
            source_view.provider <> target_table.provider OR
            source_view.datePublished <> target_table.datePublished
        )
        THEN UPDATE SET *
        WHEN NOT MATCHED THEN INSERT *
    """)


StatementMeta(, dfa0c094-e2a2-46b6-a9e3-395c9836bec4, 13, Finished, Available)